In [150]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import requests
import time
import json
import keras
import warnings
import random
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import RMSprop, Adam, Adadelta, SGD
from keras import backend as K

# Display preference
warnings.filterwarnings('ignore')

In [110]:
def scroll(driver, timeout):
    
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
    
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait
        time.sleep(timeout)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        if new_height == last_height:
            
            break
        
        last_height = new_height   

In [111]:
# Set options
options = Options()
options.set_preference('permissions.default.image', 2)
options.set_preference('dom.ipc.plugins.enabled.libflashplayer.so', False)

def all_links(url, action, gender=0):
    
    # Setup driver
    driver = webdriver.Firefox(options=options, executable_path='D:/Users/bengriffith/Desktop/geckodriver.exe')

    # Wait before throwing an exception
    driver.implicitly_wait(20)

    # Open the page
    driver.get(url)
    
    # Start scrolling
    scroll(driver, 10)
    
    # Beautiful Soup parses page_source
    soup = BeautifulSoup(driver.page_source, 'lxml')
    
    # Close the driver
    driver.close()

    if action == 1:
    
        # Empty dictionary to store m from page source
        get_m = {}
    
        # Counter for get_m dictionary
        get_m_counter = 0

        # Looping through all the a elements in the page source
        for link in soup.find_all('a', {'class': 'iusc'}):
    
            # Add values to dictionary
            get_m[get_m_counter] = link.get('m')
        
            # Increment counter
            get_m_counter += 1
        
        # Convert from string to dictionary
        for key, value in get_m.items():
            get_m[key] = json.loads(value)
        
        # Empty list to store image links
        links = []
    
        # Loop through get_m dictionary and add image links to links list
        for value in get_m.values():
            links.append([value['murl'], gender])
        
        return links
    else:
        return soup

In [112]:
# List of names to search
names = []

In [113]:
# Popular names url
name_url = 'https://www.ssa.gov/oact/babynames/decades/century.html'

# Beautiful Soup object
soup_name = all_links(name_url, 0)

# Get td tag
td_tag = soup_name.select('.t-stripe > tbody > tr > td')

# Add name to search names
names += [name.get_text().lower() for name in td_tag if name.get_text().isalpha()]

In [114]:
# List for male names
male_names = []

# List for female names
female_names = []

for index, name in enumerate(names, 0):
    if index % 2 == 0:
        male_names.append([name, 0])
    else:
        female_names.append([name, 1])

In [115]:
# Get image links for each name
def get_links(names):
    
    image_links_name = []
    
    for index in range(len(names)):
        
        # Update url
        url = 'https://www.bing.com/images/search?q=' + names[index][0] + '&qs=n&form=QBIR&qft=%20filterui%3Alicense-L2_L3_L4_L5_L6_L7%20filterui%3Aface-face&sp=-1&pq=' + names[index][0] + '&sc=8-9&sk=&cvid=8C2D8EC9A35A4948980A61669549C180'
    
        # Add image links
        image_links_name += all_links(url, 1, names[index][1])
        
    return image_links_name

In [116]:
# List for image links
image_links = []

In [117]:
# Add male name image links
image_links += get_links(male_names)

In [147]:
# Add female name image links
image_links += get_links(female_names)

In [151]:
# Shuffle image links list
random.shuffle(image_links)

In [270]:
# Header to accompany get request
headers = {'user-agent': 'scraper - school project (bengriffith@outlook.com)'}

# Counter to be used in image file name
image_counter = 0

# Counter for exceptions occurred
exception_counter = 0

# Write images to disk
for image_link in image_links[:5000]:
    with open('images/face_' + str(image_counter) + '_' + str(image_link[1]) + '.png', 'wb') as f:
        try:
            response = requests.get(image_link[0], headers=headers)
            
            if response.ok:
                f.write(response.content)
        except:
            exception_counter += 1
            continue
    image_counter += 1
    
print('There were {} files removed due to a Request / Response Exception'.format(exception_counter))

There were 8 files removed due to a Request / Response Exception


In [271]:
images_directory = 'images/'

In [272]:
files_removed = 0

for filename in os.listdir(images_directory)[1:]:
    try:
        image = cv2.imread(images_directory + filename)
        (h, w) = image.shape[:2]
    except AttributeError:
        files_removed += 1
        os.remove(images_directory + filename)
        
print('There were {} files removed due to AttributeError'.format(files_removed))

There were 56 files removed due to AttributeError


In [273]:
prototxt = 'supporting/deploy.prototxt'
model = 'supporting/res10_300x300_ssd_iter_140000.caffemodel'

In [274]:
# List to store detected face
data = []

# List to store labels
labels = []

In [275]:
for filename in os.listdir(images_directory)[1:]:
    image_path = images_directory + filename
    
    
    net = cv2.dnn.readNetFromCaffe(prototxt, model)

    image = cv2.imread(image_path)

    (h, w) = image.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(image, (10, 10)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    
    net.setInput(blob)
    detections = net.forward()
    
    for i in range(detections.shape[2]):
    
        confidence = detections[0, 0, i, 2]
    
        if confidence > 0.3:
        
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype('int')
        
            #text = '{:.2f}%'.format(confidence * 100)
            #y = startY - 10 if startY - 10 > 10 else startY + 10
            #cv2.rectangle(image, (startX, startY), (endX, endY), (0, 0, 255), 2)
            #cv2.putText(image, text, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
        
            face = image[startY:endY, startX:endX]
            face = cv2.resize(face, (32, 32)).flatten()
            data.append(face)
            labels.append(float(filename[filename.find('.') - 1]))
        else:
            continue

In [276]:
# Numpy arrays
data = np.array(data, dtype='float') / 255.0
labels = np.array(labels)

In [277]:
labels.shape

(3811,)

In [278]:
# Split into train and test datasets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.25, random_state=465)

In [279]:
X_train.shape

(2858, 3072)

In [280]:
img_rows, img_cols = 32, 32
if K.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 3, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 3, img_rows, img_cols)
    input_shape = (3, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 3)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 3)
    input_shape = (img_rows, img_cols, 3)

In [281]:
K.image_data_format()

'channels_last'

In [296]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.20))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.50))
model.add(Dense(1, activation='softmax'))

In [297]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [298]:
(unique, counts) = np.unique(labels, return_counts=True)

In [299]:
counts

array([1940, 1871], dtype=int64)

In [300]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=32)
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 2858 samples, validate on 953 samples
Epoch 1/20
2858/2858 [==============================] - 22s 8ms/step - loss: 8.2055 - accuracy: 0.4853 - val_loss: 7.8457 - val_accuracy: 0.5079
Epoch 2/20
2858/2858 [==============================] - 21s 7ms/step - loss: 8.2055 - accuracy: 0.4853 - val_loss: 7.8457 - val_accuracy: 0.5079
Epoch 3/20
2858/2858 [==============================] - 21s 7ms/step - loss: 8.2055 - accuracy: 0.4853 - val_loss: 7.8457 - val_accuracy: 0.5079
Epoch 4/20
2858/2858 [==============================] - 21s 7ms/step - loss: 8.2055 - accuracy: 0.4853 - val_loss: 7.8457 - val_accuracy: 0.5079
Epoch 5/20
2858/2858 [==============================] - 23s 8ms/step - loss: 8.2055 - accuracy: 0.4853 - val_loss: 7.8457 - val_accuracy: 0.5079
Epoch 6/20
2858/2858 [==============================] - 29s 10ms/step - loss: 8.2055 - accuracy: 0.4853 - val_loss: 7.8457 - val_accuracy: 0.5079
Epoch 7/20
2858/2858 [==============================] - 25s 9ms/step - loss: 8.205

In [ ]:
# Train neural network to recognize whether male or female

# If female then choose female GAN

# If male then choose male GAN

# Will need to create / train female and male GANs